In [1]:
# coding=utf-8
import pandas as pd
import csv  # Import csv
import tweepy
import re
import string
import nltk
import pandas as pd
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np


In [2]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-

import codecs
data = pd.read_csv('Multi label dataset.csv', header = None, encoding='utf-8', engine='python')
data.columns = ['ID','USER','FULL_TEXT','CREATED_AT','LIKE COUNT','FOLLOWERS COUNT','RETWEET COUNT','POLITICS','CRIME','ENTERTAINMENT','SPORT','OTHER'] 
data = data.dropna()
data.head()

,ID,USER,FULL_TEXT,CREATED_AT,LIKE COUNT,FOLLOWERS COUNT,RETWEET COUNT,POLITICS,CRIME,ENTERTAINMENT,SPORT,OTHER
1,35.0,1.43513E+18,පැරාලිම්පික් පදක්කම් ගත්තු දෙන්නට ජපනෙන් වාහන ...,9/7/2021 6:55,39,4175,4,0,1,0,1,1
3,36.0,1.43513E+18,පැරාලිම්පික් ක්‍රීඩා උළෙලේදී මව්බිමට ජයග්‍රහණ ...,9/7/2021 6:43,19,117259,6,1,1,0,0,0
5,37.0,1.4351E+18,අද තරගයට මිනෝද් බානුක වෙනුවට කඩුලු රකින පිතිකර...,9/7/2021 4:45,75,3126,1,0,0,1,1,0
7,38.0,1.43508E+18,ටෝකියෝ පැරාලිම්පික් ක්‍රීඩා උළෙලේදී රටට කීර්ති...,9/7/2021 3:33,0,57,0,0,1,0,0,0
9,39.0,1.43488E+18,එළඹෙන එක්දින ලෝක කුසලාන ක්‍රිකට් තරඟාවලිය සදහා...,9/6/2021 14:01,0,116,0,1,0,0,0,1


In [3]:
def clean_text(text):
    tokens = nltk.word_tokenize(text)
    regex = re.compile(u'[^\u0D80-\u0DFF]', re.UNICODE)
    tokens = [regex.sub('', w) for w in tokens]
    tokens = " ".join([c for c in tokens if c not in string.punctuation])
    return tokens

In [4]:
data['cleaned_msg'] = data['FULL_TEXT'].apply(lambda x: clean_text(x))
data.head()

,ID,USER,FULL_TEXT,CREATED_AT,LIKE COUNT,FOLLOWERS COUNT,RETWEET COUNT,POLITICS,CRIME,ENTERTAINMENT,SPORT,OTHER,cleaned_msg
1,35.0,1.43513E+18,පැරාලිම්පික් පදක්කම් ගත්තු දෙන්නට ජපනෙන් වාහන ...,9/7/2021 6:55,39,4175,4,0,1,0,1,1,පැරාලිම්පික් පදක්කම් ගත්තු දෙන්නට ජපනෙන් වාහන ...
3,36.0,1.43513E+18,පැරාලිම්පික් ක්‍රීඩා උළෙලේදී මව්බිමට ජයග්‍රහණ ...,9/7/2021 6:43,19,117259,6,1,1,0,0,0,පැරාලිම්පික් ක්රීඩා උළෙලේදී මව්බිමට ජයග්රහණ ගෙ...
5,37.0,1.4351E+18,අද තරගයට මිනෝද් බානුක වෙනුවට කඩුලු රකින පිතිකර...,9/7/2021 4:45,75,3126,1,0,0,1,1,0,අද තරගයට මිනෝද් බානුක වෙනුවට කඩුලු රකින පිතිකර...
7,38.0,1.43508E+18,ටෝකියෝ පැරාලිම්පික් ක්‍රීඩා උළෙලේදී රටට කීර්ති...,9/7/2021 3:33,0,57,0,0,1,0,0,0,ටෝකියෝ පැරාලිම්පික් ක්රීඩා උළෙලේදී රටට කීර්තිය...
9,39.0,1.43488E+18,එළඹෙන එක්දින ලෝක කුසලාන ක්‍රිකට් තරඟාවලිය සදහා...,9/6/2021 14:01,0,116,0,1,0,0,0,1,එළඹෙන එක්දින ලෝක කුසලාන ක්රිකට් තරඟාවලිය සදහා ...


In [5]:
############################ Stop Word removing Method 2 - using predefined stop words ####################################

In [6]:
stopwordsreader = open('StopWords', 'r', encoding='utf-8')
reader = csv.reader(stopwordsreader)

swords = []
for row in reader:
    swords.append(row[0])
print(swords)
    

['සහ', 'සමග', 'සමඟ', 'අහා', 'ආහ්', 'ආ', 'ඕහෝ', 'අනේ', 'අඳෝ', 'අපොයි', 'අපෝ', 'අයියෝ', 'ආයි', 'ඌයි', 'චී', 'චිහ්', 'චික්', 'හෝ\u200d', 'දෝ', 'දෝහෝ', 'මෙන්', 'සේ', 'වැනි', 'බඳු', 'වන්', 'අයුරු', 'අයුරින්', 'ලෙස', 'වැඩි', 'ශ්\u200dරී', 'හා', 'ය', 'නිසා', 'නිසාවෙන්', 'බවට', 'බව', 'බවෙන්', 'නම්', 'වැඩි', 'සිට', 'දී', 'මහා', 'මහ', 'පමණ', 'පමණින්', 'පමන', 'වන', 'විට', 'විටින්', 'මේ', 'මෙලෙස', 'මෙයින්', 'ඇති', 'ලෙස', 'සිදු', 'වශයෙන්', 'යන', 'සඳහා', 'මගින්', 'හෝ\u200d', 'ඉතා', 'ඒ', 'එම', 'ද', 'අතර', 'විසින්', 'සමග', 'පිළිබඳව', 'පිළිබඳ', 'තුළ', 'බව', 'වැනි', 'මහ', 'මෙම', 'මෙහි', 'මේ', 'වෙත', 'වෙතින්', 'වෙතට', 'වෙනුවෙන්', 'වෙනුවට', 'වෙන', 'ගැන', 'නෑ', 'අනුව', 'නව', 'පිළිබඳ', 'විශේෂ', 'දැනට', 'එහෙන්', 'මෙහෙන්', 'එහේ', 'මෙහේ', 'ම', 'තවත්', 'තව ', 'සහ', 'දක්වා', 'ට', 'ගේ', 'එ', 'ක', 'ක්', 'බවත්', 'බවද', 'මත', 'ඇතුලු', 'ඇතුළු', 'මෙසේ', 'වඩා', 'වඩාත්ම', 'නිති', 'නිතිත්', 'නිතොර', 'නිතර', 'ඉක්බිති', 'දැන්', 'යලි', 'පුන', 'ඉතින්', 'සිට', 'සිටන්', 'පටන්', 'තෙක්', 'දක්වා', 'සා', 'තාක්', 'තුවක්', 'පවා', 'ද

In [7]:
def remove_stop_words(text):
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word not in swords]
    tokens = " ".join([c for c in tokens if c not in string.punctuation])
    return tokens

In [8]:
data['remove_stopword'] = data['cleaned_msg'].apply(lambda x: remove_stop_words(x))
data.head()

,ID,USER,FULL_TEXT,CREATED_AT,LIKE COUNT,FOLLOWERS COUNT,RETWEET COUNT,POLITICS,CRIME,ENTERTAINMENT,SPORT,OTHER,cleaned_msg,remove_stopword
1,35.0,1.43513E+18,පැරාලිම්පික් පදක්කම් ගත්තු දෙන්නට ජපනෙන් වාහන ...,9/7/2021 6:55,39,4175,4,0,1,0,1,1,පැරාලිම්පික් පදක්කම් ගත්තු දෙන්නට ජපනෙන් වාහන ...,පැරාලිම්පික් පදක්කම් ගත්තු දෙන්නට ජපනෙන් වාහන ...
3,36.0,1.43513E+18,පැරාලිම්පික් ක්‍රීඩා උළෙලේදී මව්බිමට ජයග්‍රහණ ...,9/7/2021 6:43,19,117259,6,1,1,0,0,0,පැරාලිම්පික් ක්රීඩා උළෙලේදී මව්බිමට ජයග්රහණ ගෙ...,පැරාලිම්පික් ක්රීඩා උළෙලේදී මව්බිමට ජයග්රහණ ගෙ...
5,37.0,1.4351E+18,අද තරගයට මිනෝද් බානුක වෙනුවට කඩුලු රකින පිතිකර...,9/7/2021 4:45,75,3126,1,0,0,1,1,0,අද තරගයට මිනෝද් බානුක වෙනුවට කඩුලු රකින පිතිකර...,අද තරගයට මිනෝද් බානුක කඩුලු රකින පිතිකරු දිනේෂ...
7,38.0,1.43508E+18,ටෝකියෝ පැරාලිම්පික් ක්‍රීඩා උළෙලේදී රටට කීර්ති...,9/7/2021 3:33,0,57,0,0,1,0,0,0,ටෝකියෝ පැරාලිම්පික් ක්රීඩා උළෙලේදී රටට කීර්තිය...,ටෝකියෝ පැරාලිම්පික් ක්රීඩා උළෙලේදී රටට කීර්තිය...
9,39.0,1.43488E+18,එළඹෙන එක්දින ලෝක කුසලාන ක්‍රිකට් තරඟාවලිය සදහා...,9/6/2021 14:01,0,116,0,1,0,0,0,1,එළඹෙන එක්දින ලෝක කුසලාන ක්රිකට් තරඟාවලිය සදහා ...,එළඹෙන එක්දින ලෝක කුසලාන ක්රිකට් තරඟාවලිය සදහා ...


In [9]:
########################################## Stemming ##################################################

In [10]:
stem_dict = open("stem_dictionary.txt", "r", encoding='utf-8')
stem_dict = map(lambda s: s.strip(), stem_dict)

stem_dictionary = {}

for s in stem_dict:
    s = s.split("\t")
    stem_dictionary[s[0]] = s[1]
    


In [11]:
def stemming_words(text):
    tokens = nltk.word_tokenize(text)
    for k, v in enumerate(tokens):
        tokens[k] = stem_dictionary.get(v, v)
    tokens = " ".join([c for c in tokens if c not in string.punctuation])
    return tokens

In [12]:
data['stemming_words'] = data['remove_stopword'].apply(lambda x: stemming_words(x))
data.head()

,ID,USER,FULL_TEXT,CREATED_AT,LIKE COUNT,FOLLOWERS COUNT,RETWEET COUNT,POLITICS,CRIME,ENTERTAINMENT,SPORT,OTHER,cleaned_msg,remove_stopword,stemming_words
1,35.0,1.43513E+18,පැරාලිම්පික් පදක්කම් ගත්තු දෙන්නට ජපනෙන් වාහන ...,9/7/2021 6:55,39,4175,4,0,1,0,1,1,පැරාලිම්පික් පදක්කම් ගත්තු දෙන්නට ජපනෙන් වාහන ...,පැරාලිම්පික් පදක්කම් ගත්තු දෙන්නට ජපනෙන් වාහන ...,පැරාලිම්පික් පදක්කම් ගත්තු දෙන්නට ජපනෙන් වාහන ...
3,36.0,1.43513E+18,පැරාලිම්පික් ක්‍රීඩා උළෙලේදී මව්බිමට ජයග්‍රහණ ...,9/7/2021 6:43,19,117259,6,1,1,0,0,0,පැරාලිම්පික් ක්රීඩා උළෙලේදී මව්බිමට ජයග්රහණ ගෙ...,පැරාලිම්පික් ක්රීඩා උළෙලේදී මව්බිමට ජයග්රහණ ගෙ...,පැරාලිම්පික් ක්රීඩා උළෙලේදී මව්බිම ජයග්රහණ ගෙන...
5,37.0,1.4351E+18,අද තරගයට මිනෝද් බානුක වෙනුවට කඩුලු රකින පිතිකර...,9/7/2021 4:45,75,3126,1,0,0,1,1,0,අද තරගයට මිනෝද් බානුක වෙනුවට කඩුලු රකින පිතිකර...,අද තරගයට මිනෝද් බානුක කඩුලු රකින පිතිකරු දිනේෂ...,අද තරඟ මිනෝද් බානුක කඩුලු රකින පිතිකරු දිනේෂ ච...
7,38.0,1.43508E+18,ටෝකියෝ පැරාලිම්පික් ක්‍රීඩා උළෙලේදී රටට කීර්ති...,9/7/2021 3:33,0,57,0,0,1,0,0,0,ටෝකියෝ පැරාලිම්පික් ක්රීඩා උළෙලේදී රටට කීර්තිය...,ටෝකියෝ පැරාලිම්පික් ක්රීඩා උළෙලේදී රටට කීර්තිය...,ටෝකියෝ පැරාලිම්පික් ක්රීඩා උළෙලේදී රටට කීර්තිය...
9,39.0,1.43488E+18,එළඹෙන එක්දින ලෝක කුසලාන ක්‍රිකට් තරඟාවලිය සදහා...,9/6/2021 14:01,0,116,0,1,0,0,0,1,එළඹෙන එක්දින ලෝක කුසලාන ක්රිකට් තරඟාවලිය සදහා ...,එළඹෙන එක්දින ලෝක කුසලාන ක්රිකට් තරඟාවලිය සදහා ...,එළඹෙන එක්දින ලෝක කුසලා ක්රිකට් තරඟ සදහා ක්රීඩා...


In [13]:
######################### Remove Duplicates from dataset #################

In [14]:
data.drop_duplicates(subset='stemming_words',keep='first',inplace = True)
data.head()

,ID,USER,FULL_TEXT,CREATED_AT,LIKE COUNT,FOLLOWERS COUNT,RETWEET COUNT,POLITICS,CRIME,ENTERTAINMENT,SPORT,OTHER,cleaned_msg,remove_stopword,stemming_words
1,35.0,1.43513E+18,පැරාලිම්පික් පදක්කම් ගත්තු දෙන්නට ජපනෙන් වාහන ...,9/7/2021 6:55,39,4175,4,0,1,0,1,1,පැරාලිම්පික් පදක්කම් ගත්තු දෙන්නට ජපනෙන් වාහන ...,පැරාලිම්පික් පදක්කම් ගත්තු දෙන්නට ජපනෙන් වාහන ...,පැරාලිම්පික් පදක්කම් ගත්තු දෙන්නට ජපනෙන් වාහන ...
3,36.0,1.43513E+18,පැරාලිම්පික් ක්‍රීඩා උළෙලේදී මව්බිමට ජයග්‍රහණ ...,9/7/2021 6:43,19,117259,6,1,1,0,0,0,පැරාලිම්පික් ක්රීඩා උළෙලේදී මව්බිමට ජයග්රහණ ගෙ...,පැරාලිම්පික් ක්රීඩා උළෙලේදී මව්බිමට ජයග්රහණ ගෙ...,පැරාලිම්පික් ක්රීඩා උළෙලේදී මව්බිම ජයග්රහණ ගෙන...
5,37.0,1.4351E+18,අද තරගයට මිනෝද් බානුක වෙනුවට කඩුලු රකින පිතිකර...,9/7/2021 4:45,75,3126,1,0,0,1,1,0,අද තරගයට මිනෝද් බානුක වෙනුවට කඩුලු රකින පිතිකර...,අද තරගයට මිනෝද් බානුක කඩුලු රකින පිතිකරු දිනේෂ...,අද තරඟ මිනෝද් බානුක කඩුලු රකින පිතිකරු දිනේෂ ච...
7,38.0,1.43508E+18,ටෝකියෝ පැරාලිම්පික් ක්‍රීඩා උළෙලේදී රටට කීර්ති...,9/7/2021 3:33,0,57,0,0,1,0,0,0,ටෝකියෝ පැරාලිම්පික් ක්රීඩා උළෙලේදී රටට කීර්තිය...,ටෝකියෝ පැරාලිම්පික් ක්රීඩා උළෙලේදී රටට කීර්තිය...,ටෝකියෝ පැරාලිම්පික් ක්රීඩා උළෙලේදී රටට කීර්තිය...
9,39.0,1.43488E+18,එළඹෙන එක්දින ලෝක කුසලාන ක්‍රිකට් තරඟාවලිය සදහා...,9/6/2021 14:01,0,116,0,1,0,0,0,1,එළඹෙන එක්දින ලෝක කුසලාන ක්රිකට් තරඟාවලිය සදහා ...,එළඹෙන එක්දින ලෝක කුසලාන ක්රිකට් තරඟාවලිය සදහා ...,එළඹෙන එක්දින ලෝක කුසලා ක්රිකට් තරඟ සදහා ක්රීඩා...


In [15]:
df = pd.DataFrame(data, columns= ['ID','stemming_words','CREATED_AT','LIKE COUNT','FOLLOWERS COUNT','RETWEET COUNT','POLITICS','CRIME','ENTERTAINMENT','SPORT','OTHER'])
df.to_csv("FinalListMultiLabledPreprocessedData.csv")